# 1. Initialize

### Libraries and Spark context

In [20]:
import pyspark
from pyspark.sql import SparkSession
from probetrip.od import OD

In [21]:
spark = SparkSession.builder.master("yarn").appName("4_origin_destination") \
        .config("spark.executor.memory", "4g").config("spark.driver.memory", "4g") \
        .config("spark.dynamicAllocation.minExecutors", "1").config("spark.dynamicAllocation.maxExecutors", "3") \
        .config("spark.sql.catalogImplementation", "hive") \
        .config("metastore.catalog.default", "hive") \
        .enableHiveSupport() \
        .getOrCreate()

24/07/16 11:26:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/07/16 11:26:54 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [27]:
spark.stop()

### Required parameter

**Trip table structure** (input table)
<table style='float:left;'>
    <tr>
        <th style='text-align: left;'>Name</th>
        <th style='text-align: left;'>Data type</th>
        <th style='text-align: left;'>Example (Stay)</th>
        <th style='text-align: left;'>Example (Move)</th>
    </tr>
    <tr>
        <td style='text-align: left;'>id</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>+1indEOKr/ikPVrJQTVjw4FGxBE</td>
        <td style='text-align: left;'>+1indEOKr/ikPVrJQTVjw4FGxBE</td>
    </tr>
    <tr>
        <td style='text-align: left;'>trip_date</td>
        <td style='text-align: left;'>date </td>
        <td style='text-align: left;'>2023-01-01</td>
        <td style='text-align: left;'>2023-01-01</td>
    </tr>
    <tr>
        <td style='text-align: left;'>trip_sequence</td>
        <td style='text-align: left;'>integer</td>
        <td style='text-align: left;'>1</td>
        <td style='text-align: left;'>1</td>
    </tr>
    <tr>
        <td style='text-align: left;'>trip_sequence</td>
        <td style='text-align: left;'>integer</td>
        <td style='text-align: left;'>1</td>
        <td style='text-align: left;'>1</td>
    </tr>
    <tr>
        <td style='text-align: left;'>mobility_type</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>STAY</td>
        <td style='text-align: left;'>MOVE</td>
    </tr>
    <tr>
        <td style='text-align: left;'>passenger</td>
        <td style='text-align: left;'>integer</td>
        <td style='text-align: left;'>0</td>
        <td style='text-align: left;'>1</td>
    </tr>
    <tr>
        <td style='text-align: left;'>total_distance (km)</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>0.0</td>
        <td style='text-align: left;'>209.57</td>
    </tr>
    <tr>
        <td style='text-align: left;'>total_time (minutes)</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>318.53</td>
        <td style='text-align: left;'>390.3</td>
    </tr>
    <tr>
        <td style='text-align: left;'>overall_speed (km/h)</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>318.53</td>
        <td style='text-align: left;'>32.22</td>
    </tr>
    <tr>
        <td style='text-align: left;'>average_speed (km/h)</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>0.0</td>
        <td style='text-align: left;'>28.8</td>
    </tr>
    <tr>
        <td style='text-align: left;'>start_time</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>00:02:49</td>
        <td style='text-align: left;'>05:21:21</td>
    </tr>
    <tr>
        <td style='text-align: left;'>end_time</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>05:21:21</td>
        <td style='text-align: left;'>11:51:39</td>
    </tr>
    <tr>
        <td style='text-align: left;'>total_points</td>
        <td style='text-align: left;'>integer</td>
        <td style='text-align: left;'>121</td>
        <td style='text-align: left;'>283</td>
    </tr>
    <tr>
        <td style='text-align: left;'>sub_district</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>Lam Phak Kut</td>
        <td style='text-align: left;'></td>
    </tr>
    <tr>
        <td style='text-align: left;'>district</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>Thanyaburi</td>
        <td style='text-align: left;'></td>
    </tr>
    <tr>
        <td style='text-align: left;'>province</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>Pathum Thani</td>
        <td style='text-align: left;'></td>
    </tr>
    <tr>
        <td style='text-align: left;'>point_list</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>[[14.032970,100.786000]]</td>
        <td style='text-align: left;'>[["05:21:21",14.03297,100.786],["05:22:21",14.0346,100.78606], ..., ["11:51:39",14.03296,100.78601]]</td>
    </tr>
    <tr>
        <td style='text-align: left;'>partition_month</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>202301</td>
        <td style='text-align: left;'>202301</td>
    </tr>
</table>

**OD table structure** (output table)
<table style='float:left;'>
    <tr>
        <th style='text-align: left;'>Name</th>
        <th style='text-align: left;'>Data type</th>
        <th style='text-align: left;'>Example</th>
    </tr>
    <tr>
        <td style='text-align: left;'>id</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>+A3arvBS15eOvdHE+E06+Ng28+E</td>
    </tr>
    <tr>
        <td style='text-align: left;'>trip_date</td>
        <td style='text-align: left;'>date</td>
        <td style='text-align: left;'>2023-01-01</td>
    </tr>
    <tr>
        <td style='text-align: left;'>trip_sequence</td>
        <td style='text-align: left;'>integer</td>
        <td style='text-align: left;'>2</td>
    </tr>
    <tr>
        <td style='text-align: left;'>origin_hour</td>
        <td style='text-align: left;'>integer</td>
        <td style='text-align: left;'>6</td>
    </tr>
    <tr>
        <td style='text-align: left;'>origin_lat</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>13.75927</td>
    </tr>
    <tr>
        <td style='text-align: left;'>origin_lon</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>100.594</td>
    </tr>
    <tr>
        <td style='text-align: left;'>origin_sub_district</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>Bang Kapi</td>
    </tr>
    <tr>
        <td style='text-align: left;'>origin_district</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>Huai Khwang</td>
    </tr>
    <tr>
        <td style='text-align: left;'>origin_province</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>Bangkok</td>
    </tr>
    <tr>
        <td style='text-align: left;'>destination_hour</td>
        <td style='text-align: left;'>integer</td>
        <td style='text-align: left;'>7</td>
    </tr>
    <tr>
        <td style='text-align: left;'>destination_lat</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>13.73948</td>
    </tr>
    <tr>
        <td style='text-align: left;'>destination_lon</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>100.60275</td>
    </tr>
    <tr>
        <td style='text-align: left;'>destination_sub_district</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>Suan Luang</td>
    </tr>
    <tr>
        <td style='text-align: left;'>destination_district</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>Suan Luang</td>
    </tr>
    <tr>
        <td style='text-align: left;'>destination_province</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>Bangkok</td>
    </tr>
    <tr>
        <td style='text-align: left;'>partition_month</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>202301</td>
    </tr>
</table>

In [ ]:
database_name = "probe_taxi"
trip_table = "trip_point"
od_table = "od_occupied"
shape_path = "dataset/tha_shapefiles/tha_admbnda_adm3_rtsd_20220121.shp"
month = '202301'

# 2. Execution

In [11]:
od = OD(spark, database_name, trip_table, od_table, shape_path)

In [12]:
od.generate_od(month)

Generate Origin-Destination on 202301 successfuly! 	 | Process time: 7.19 mins
